# Setup

In [2]:
import time
beginning_time = time.time()

import sys
sys.path.append("../Code/")
from utils import performance

import os
DATA_DIR = os.path.join('..', 'Data')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [6]:
def timtime(t):
    print(">",int(time.time()- t), "seconds elapsed")

# Load data

In [7]:
# sparse might be important?

develop = False
        
train_X = pd.concat([pd.read_pickle(os.path.join(DATA_DIR, 'train_X_1')), 
                     pd.read_pickle(os.path.join(DATA_DIR, 'train_X_2'))])
train_y = pd.read_pickle(os.path.join(DATA_DIR, 'train_y'))
valid_X = pd.read_pickle(os.path.join(DATA_DIR, 'valid_X'))
valid_y = pd.read_pickle(os.path.join(DATA_DIR, 'valid_y'))

if develop:
    train_X = train_X[0:10000]
    train_y = train_y[0:10000]
    valid_X = valid_X[0:1000]
    valid_y = valid_y[0:1000]

In [9]:
test_X = pd.read_pickle(os.path.join(DATA_DIR, 'test_X'))

In [8]:
test_budget=625000

In [10]:
train_budget = (test_budget*len(train_X))//len(test_X)

In [11]:
train_budget

5008201

In [12]:
sys.getsizeof(train_X)/1000000000

3.29641026

In [13]:
from scipy.sparse import csr_matrix
train_X = csr_matrix(train_X) 
valid_X = csr_matrix(valid_X) 

In [15]:
sys.getsizeof(train_X)/1000000000

5.6e-08

# CTR estimation

### GBDT

In [16]:
from xgboost import XGBRegressor
# for some reason the XGBClassifier was not working

gbdt_crt_model = XGBRegressor(max_depth=10, n_estimators=75, random_state=0, 
                        max_delta_step=1, objective='binary:logistic', learning_rate=0.1, 
                        scale_pos_weight=1)

gbdt_crt_model.fit(train_X, train_y.click, eval_metric="logloss", eval_set=[(valid_X, valid_y.click)], 
                   verbose=True, early_stopping_rounds=7)

[0]	validation_0-logloss:0.644463
Will train until validation_0-logloss hasn't improved in 7 rounds.
[1]	validation_0-logloss:0.598272
[2]	validation_0-logloss:0.554555
[3]	validation_0-logloss:0.513281
[4]	validation_0-logloss:0.474409
[5]	validation_0-logloss:0.437887
[6]	validation_0-logloss:0.403651
[7]	validation_0-logloss:0.371632
[8]	validation_0-logloss:0.341752
[9]	validation_0-logloss:0.313926
[10]	validation_0-logloss:0.288066
[11]	validation_0-logloss:0.26408
[12]	validation_0-logloss:0.241872
[13]	validation_0-logloss:0.221348
[14]	validation_0-logloss:0.20241
[15]	validation_0-logloss:0.184964
[16]	validation_0-logloss:0.168916
[17]	validation_0-logloss:0.154174
[18]	validation_0-logloss:0.140649
[19]	validation_0-logloss:0.128257
[20]	validation_0-logloss:0.116915
[21]	validation_0-logloss:0.106545
[22]	validation_0-logloss:0.097057
[23]	validation_0-logloss:0.088398
[24]	validation_0-logloss:0.080501
[25]	validation_0-logloss:0.073307
[26]	validation_0-logloss:0.066757


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=1,
       max_depth=10, min_child_weight=1, missing=None, n_estimators=75,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [27]:
##### save both models
import pickle
pkl_filename = "../Models/tim_xgb_click.pkl"  

with open(pkl_filename, 'wb') as file:  
    pickle.dump(gbdt_crt_model, file)
    
##### save predicted prices and predicted probabilities